In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

### Read the matching result into a dataframe

In [3]:
mr_file = "../data/mr.txt"

In [4]:
df = pd.read_csv(mr_file,sep=";")

In [5]:
def extract_cpath(cpath):
    if (cpath==''):
        return []
    return [int(s) for s in cpath.split(',')]

### Extract cpath as list of int

In [6]:
df["cpath_list"] = df.apply(lambda row: extract_cpath(row.cpath), axis=1)

### Extract all the edges matched

In [7]:
all_edge_ids = np.unique(np.hstack(df.cpath_list)).tolist()

### Read road network with geopandas

In [8]:
network_gdf = gpd.read_file("../data/edges.shp")
network_gdf.id = network_gdf.id.astype(int)
network_gdf.head()

,_uid_,id,source,target,cost,x1,y1,x2,y2,geometry
0,1,1,1,2,1.0,2.0,1.0,2.0,0.0,"LINESTRING (2 1, 2 0)"
1,2,2,2,1,1.0,2.0,0.0,2.0,1.0,"LINESTRING (2 0, 2 1)"
2,3,3,3,1,1.0,3.0,1.0,2.0,1.0,"LINESTRING (3 1, 2 1)"
3,4,4,4,3,1.0,4.0,1.0,3.0,1.0,"LINESTRING (4 1, 3 1)"
4,5,5,1,5,1.0,2.0,1.0,2.0,2.0,"LINESTRING (2 1, 2 2)"


In [9]:
edges_df = network_gdf[network_gdf.id.isin(all_edge_ids)].reset_index()

In [10]:
edges_df["points"] = edges_df.apply(lambda row: len(row.geometry.coords), axis=1)

In [11]:
edges_df.head()

,index,_uid_,id,source,target,cost,x1,y1,x2,y2,geometry,points
0,1,2,2,2,1,1.0,2.0,0.0,2.0,1.0,"LINESTRING (2 0, 2 1)",2
1,2,3,3,3,1,1.0,3.0,1.0,2.0,1.0,"LINESTRING (3 1, 2 1)",2
2,3,4,4,4,3,1.0,4.0,1.0,3.0,1.0,"LINESTRING (4 1, 3 1)",2
3,4,5,5,1,5,1.0,2.0,1.0,2.0,2.0,"LINESTRING (2 1, 2 2)",2
4,7,8,8,7,8,1.0,0.0,2.0,1.0,2.0,"LINESTRING (0 2, 1 2)",2


### Export the matched edges to a csv file

In [12]:
edges_df[["id","source","target","geometry","points"]].to_csv("a.csv",sep=";",index=False)